# Advanced Pandas features

## Hierarchical indexing

In [157]:
oecd

,Country,Year,Population,GDP,Debt
0,Australia,2000,19153380,416923318470,11
1,Australia,2001,19413240,380427712783,10
...,...,...,...,...,...
68,United States,2008,304177400,14296900000000,40
69,United States,2009,306656300,14043900000000,54


In [158]:
# Pandas can use the same array-oriented boolean indexing operators like | and & as NumPy:
idx = (oecd.Country == "Australia") | (oecd.Country == "France")
subset = oecd.loc[idx, ["Year", "Country", "Debt", "GDP"]]
subset

,Year,Country,Debt,GDP
0,2000,Australia,11,416923318470
1,2001,Australia,10,380427712783
...,...,...,...,...
28,2008,France,53,2831794060131
29,2009,France,61,2624504232173


In [159]:
pivoted = subset.pivot(index="Year", columns="Country")
pivoted

Debt                   GDP               
Country Australia France      Australia         France
Year                                                  
2000           11     47   416923318470  1326334438917
2001           10     48   380427712783  1338302550336
...           ...    ...            ...            ...
2008            5     53  1039415095377  2831794060131
2009            8     61   924843128521  2624504232173

[10 rows x 4 columns]

The data frame has two rows: the inner row for 'Country' and the outer row for other variables.

In [105]:
pivoted["Debt"] # Extract a DataFrame from the "Debt" column.

Country,Australia,France
Year,,
2000,11.361,47.417
2001,9.552,48.346
...,...,...
2008,4.922,53.406
2009,8.195,61.231


In [106]:
# Two nested column labels. Extracts a Series:
pivoted["Debt","Australia"]

Year
2000    11.361
2001     9.552
         ...  
2008     4.922
2009     8.195
Name: (Debt, Australia), Length: 10, dtype: float64

In [107]:
# This is almost the same as pivoted["Debt","Australia"], except the label name:
pivoted["Debt"]["Australia"]

Year
2000    11.361
2001     9.552
         ...  
2008     4.922
2009     8.195
Name: Australia, Length: 10, dtype: float64

In [108]:
# Swap the first level and the second level in column labels.
swapped = pivoted.swaplevel(0,1,axis=1)
swapped

Country,Australia,France,Australia,France
,Debt,Debt,GDP,GDP
Year,,,,
2000,11.361,47.417,416923318470,1326334438917
2001,9.552,48.346,380427712783,1338302550336
...,...,...,...,...
2008,4.922,53.406,1039415095377,2831794060131
2009,8.195,61.231,924843128521,2624504232173


## More on table pivoting: stack and unstack

In [109]:
smalltable = abalone[['diameter', 'shell weight', 'shucked weight']]  # for easy display
smalltable

,diameter,shell weight,shucked weight
0,0.365,0.150,0.225
1,0.265,0.070,0.100
...,...,...,...
4175,0.485,0.296,0.531
4176,0.555,0.495,0.946


In [110]:
smalltable.stack()   # Returns a Series (squished down) with a hierarchical index

0     diameter          0.365
      shell weight      0.150
                        ...  
4176  shell weight      0.495
      shucked weight    0.946
Length: 12531, dtype: float64

In [111]:
smalltable.stack().unstack() # DataFrame.unstack() restores a stacked table back to the original table.

,diameter,shell weight,shucked weight
0,0.365,0.150,0.225
1,0.265,0.070,0.100
...,...,...,...
4175,0.485,0.296,0.531
4176,0.555,0.495,0.946


### More advanced pivoting

Now, we are going to create the following table from the *oecd* table using pivoting.

<img src="extras/pivoted.png" />

In the table, each row shows the means of debt, GDP, population and total debt of all the countries. This is how to generate the summary table.

In [112]:
oecd

,Country,Year,Population,GDP,Debt
0,Australia,2000,19153380,416923318470,11.361
1,Australia,2001,19413240,380427712783,9.552
...,...,...,...,...,...
68,United States,2008,304177400,14296900000000,40.183
69,United States,2009,306656300,14043900000000,53.573


In [113]:
# We will extract a subset to make pivoting clearer
subset1 = oecd['Country'] <= 'G'
subset2 = oecd['Year'] >= 2006
smalldata = oecd[subset1 & subset2]
smalldata

,Country,Year,Population,GDP,Debt
6,Australia,2006,20697880,749316412099,5.760
7,Australia,2007,21015040,856816361781,5.181
...,...,...,...,...,...
28,France,2008,62304320,2831794060131,53.406
29,France,2009,62636430,2624504232173,61.231


In [114]:
# Pivot 'smalldata' DataFrame by 'Year' and 'Country'. Country names become the label for the inner level of the column index.
pivoted2 = smalldata.pivot(index="Year", columns="Country")
pivoted2

Population                       ...        Debt                
Country  Australia    Canada    France   ...   Australia  Canada  France
Year                                     ...                            
2006      20697880  32576070  61597490   ...       5.760  27.934  52.131
2007      21015040  32929730  61965050   ...       5.181  25.183  52.118
2008      21499000  33315980  62304320   ...       4.922  28.642  53.406
2009      21955000  33720180  62636430   ...       8.195  35.716  61.231

[4 rows x 9 columns]

In [115]:
# The stack() returns a DataFrame with an index with a new inner-most level of row labels.
pivoted2.stack()

Population            GDP    Debt
Year Country                                     
2006 Australia    20697880   749316412099   5.760
     Canada       32576070  1278610846645  27.934
...                    ...            ...     ...
2009 Canada       33720180  1336067710612  35.716
     France       62636430  2624504232173  61.231

[12 rows x 3 columns]

In [116]:
# The stack(level=0) returns a DataFrame with an index with row labels of level=0.
pivoted2.stack(level=0)

Country          Australia     Canada     France
Year                                            
2006 Population  2.070e+07  3.258e+07  6.160e+07
     GDP         7.493e+11  1.279e+12  2.256e+12
...                    ...        ...        ...
2009 GDP         9.248e+11  1.336e+12  2.625e+12
     Debt        8.195e+00  3.572e+01  6.123e+01

[12 rows x 3 columns]

In [117]:
# Then calculate mean for each row:
pivoted2.stack(0).mean(axis=1)

Year            
2006  Population    3.829e+07
      GDP           1.428e+12
                      ...    
2009  GDP           1.628e+12
      Debt          3.505e+01
Length: 12, dtype: float64

In [118]:
# Return a reshaped DataFrame with a new inner-most level of column labels.
pivoted2.stack(0).mean(axis=1).unstack()

,Population,GDP,Debt
Year,,,
2006,3.829e+07,1.428e+12,28.608
2007,3.864e+07,1.621e+12,27.494
2008,3.904e+07,1.790e+12,28.990
2009,3.944e+07,1.628e+12,35.047


## Saving Pandas objects to HDF5 files

In [119]:
# We can save multiple Series or DataFrames to an HDF file.
hdf = pd.HDFStore("all_data.h5", "w")
hdf["oecd"] = oecd
hdf["cars"] = cars
hdf["abalone"] = abalone
hdf["pivoted"] = pivoted
# etc...
hdf # List variables and their types stored in the file.

<class 'pandas.io.pytables.HDFStore'>
File path: all_data.h5
/abalone            frame        (shape->[4177,10])
/cars               frame        (shape->[392,9])  
/oecd               frame        (shape->[70,5])   
/pivoted            frame                          

In [120]:
hdf["abalone"]   # The same as the 'data' DataFrame.

,sex,length,diameter,...,shell weight,rings,weight to diameter ratio
0,M,0.455,0.365,...,0.150,15,1.408
1,M,0.350,0.265,...,0.070,7,0.851
...,...,...,...,...,...,...,...
4175,F,0.625,0.485,...,0.296,10,2.257
4176,M,0.710,0.555,...,0.495,12,3.511


In [121]:
hdf.close()

## Worked example

Which country was the most efficient at the 2012 London Olympic Games, in terms of population to Gold medal count?

In [4]:
%run shortcut_imports.ipy

In [37]:
import pandas as pd
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 5)
pd.set_option('display.precision', 3)

In [38]:
medals = pd.read_csv("data/olympics2012.csv")
medals

,Country,Gold,Silver,Bronze
0,Afghanistan,0,0,1
1,Albania,0,0,0
...,...,...,...,...
202,Zambia,0,0,0
203,Zimbabwe,0,0,0


In [39]:
countries = pd.read_csv("data/country_populations_by_year.csv")
countries.head()

,Country Name,Country Code,...,2009,2010
0,Afghanistan,AFG,...,3.344e+07,3.439e+07
1,Albania,ALB,...,3.193e+06,3.204e+06
2,Algeria,DZA,...,3.495e+07,3.547e+07
3,American Samoa,ASM,...,6.731e+04,6.842e+04
4,Andorra,AND,...,8.368e+04,8.486e+04


To answer this question, we join these two datasets together. Before we do that, we need to set the index, so that the datasets can be joined.

In [40]:
medals.set_index("Country", inplace=True)
medals

,Gold,Silver,Bronze
Country,,,
Afghanistan,0,0,1
Albania,0,0,0
...,...,...,...
Zambia,0,0,0
Zimbabwe,0,0,0


In [41]:
countries.set_index("Country Name", inplace=True)
countries

,Country Code,1960,...,2009,2010
Country Name,,,,,
Afghanistan,AFG,9671046,...,3.344e+07,3.439e+07
Albania,ALB,1610565,...,3.193e+06,3.204e+06
...,...,...,...,...,...
Zambia,ZMB,3044733,...,1.272e+07,1.293e+07
Zimbabwe,ZWE,3752373,...,1.247e+07,1.257e+07


Now for the join...

In [42]:
medals.join(countries)

,Gold,Silver,...,2009,2010
Country,,,,,
Afghanistan,0,0,...,3.344e+07,3.439e+07
Albania,0,0,...,3.193e+06,3.204e+06
...,...,...,...,...,...
Zambia,0,0,...,1.272e+07,1.293e+07
Zimbabwe,0,0,...,1.247e+07,1.257e+07


This is a bit too much data, let's reduce down to just 2010's data

In [43]:
fulldata = medals.join(countries["2010"])
fulldata

,Gold,Silver,Bronze,2010
Country,,,,
Afghanistan,0,0,1,3.439e+07
Albania,0,0,0,3.204e+06
...,...,...,...,...
Zambia,0,0,0,1.293e+07
Zimbabwe,0,0,0,1.257e+07


In [44]:
fulldata.columns

Index(['Gold', 'Silver', 'Bronze', '2010'], dtype='object')

In [45]:
# Let's fix the column name
fulldata.rename(columns={'2010':"Population"}, inplace=True)
fulldata

,Gold,Silver,Bronze,Population
Country,,,,
Afghanistan,0,0,1,3.439e+07
Albania,0,0,0,3.204e+06
...,...,...,...,...
Zambia,0,0,0,1.293e+07
Zimbabwe,0,0,0,1.257e+07


In [46]:
fulldata["Gold per million"] = fulldata["Gold"] / (fulldata["Population"] / 10**6)

In [47]:
fulldata.sort_values("Gold per million", ascending=False)[:5]

,Gold,Silver,Bronze,Population,Gold per million
Country,,,,,
Grenada,1,0,0,1.045e+05,9.571
Jamaica,4,4,4,2.702e+06,1.480
New Zealand,6,2,5,4.368e+06,1.374
Hungary,8,4,5,1.001e+07,0.799
Trinidad and Tobago,1,0,3,1.341e+06,0.745


Some more on this join:

### 1. You don't need to actually create fulldata:

In [48]:
gold_per_million = medals["Gold"] / (countries["2010"] / 10**6)
gold_per_million.sort_values(ascending=False)[:5]

Grenada                9.571
Jamaica                1.480
New Zealand            1.374
Hungary                0.799
Trinidad and Tobago    0.745
dtype: float64

### 2. You don't need to reindex the data

Although its better if you do

In [49]:
medals = pd.read_csv("data/olympics2012.csv")
countries = pd.read_csv("data/country_populations_by_year.csv")

In [50]:
medals.merge(countries, left_on="Country", right_on="Country Name")[["Country", "Gold", "2010"]]

,Country,Gold,2010
0,Afghanistan,0,3.439e+07
1,Albania,0,3.204e+06
...,...,...,...
168,Zambia,0,1.293e+07
169,Zimbabwe,0,1.257e+07


### 3. There is more than one type of join

In [51]:
medals = pd.read_csv("data/olympics2012.csv", index_col="Country")
countries = pd.read_csv("data/country_populations_by_year.csv", index_col="Country Name")

In [52]:
len(medals.join(countries["2010"], how='left'))

204

In [53]:
len(medals.join(countries["2010"], how='right'))

211

In [54]:
len(medals.join(countries["2010"], how='inner'))

170

In [55]:
len(medals.join(countries["2010"], how='outer'))

245

Examine the different results to find out the differences.

In [56]:
inner = medals.join(countries["2010"], how='inner')
outer = medals.join(countries["2010"], how='outer')

outer.index.difference(inner.index)

Index(['American Samoa', 'American Virgin Islands', 'Bahamas', 'Bahamas, The',
       'British Virgin Islands', 'Burma (Myanmar)', 'Channel Islands', 'China',
       'China, People's Republic of', 'Congo', 'Congo, Dem. Rep.',
       'Congo, Rep.', 'Congo, the Democratic Republic of the', 'Cook Islands',
       'DPR Korea', 'Egypt', 'Egypt, Arab Rep.', 'Faeroe Islands',
       'Former Yugoslav Republic of Macedonia', 'French Polynesia', 'Gambia',
       'Gambia, The', 'Gaza Strip/Palestine/West Bank', 'Gibraltar',
       'Great Britain', 'Greenland', 'Hong Kong', 'Hong Kong SAR, China',
       'Iran', 'Iran, Islamic Rep.', 'Isle of Man', 'Korea, Dem. Rep.',
       'Korea, Rep.', 'Korea, Republic of', 'Kosovo', 'Kyrgyz Republic',
       'Kyrgyzstan', 'Lao PDR', 'Laos', 'Macao SAR, China', 'Macedonia, FYR',
       'Mayotte', 'Micronesia', 'Micronesia, Fed. Sts.', 'Myanmar', 'Nauru',
       'New Caledonia', 'Northern Mariana Islands', 'Russia',
       'Russian Federation', 'Samoa, American

**QUESTION**: Why are there differences between these joins?

## Sorting indices

Our datasets here are sorted, which makes joining much easier - it takes less time. However, not all datasets will come like this. Let's shuffle the index and then resort it.

In [57]:
import random
shuffled_index = list(medals.index.values)  # shuffle is an in-place operation, so we must copy the values!
random.shuffle(shuffled_index)

In [61]:
medals_shuffled = medals.loc[shuffled_index]
medals_shuffled.head()

,Gold,Silver,Bronze
Country,,,
Dominica,0,0,0
Peru,0,0,0
Seychelles,0,0,0
Uganda,1,0,0
Senegal,0,0,0


In [62]:
medals_shuffled.sort_index()

,Gold,Silver,Bronze
Country,,,
Afghanistan,0,0,1
Albania,0,0,0
...,...,...,...
Zambia,0,0,0
Zimbabwe,0,0,0


### Exercise

Time the difference between joining shuffled and sorted dataframes using %%timeit

## Shifting the index

Series and DataFrames provide a set of methods of the form "compare this row to the previous one". These are `shift`, `pct_change`, and `diff`. First, let's create a dataset to examine this.

In [63]:
countries

,Country Code,1960,...,2009,2010
Country Name,,,,,
Afghanistan,AFG,9671046,...,3.344e+07,3.439e+07
Albania,ALB,1610565,...,3.193e+06,3.204e+06
...,...,...,...,...,...
Zambia,ZMB,3044733,...,1.272e+07,1.293e+07
Zimbabwe,ZWE,3752373,...,1.247e+07,1.257e+07


In [64]:
# To get help: countries.drop?

In [65]:
populations = countries.drop("Country Code", axis=1).transpose()

In [66]:
populations["United States"]

1960    1.807e+08
1961    1.837e+08
          ...    
2009    3.070e+08
2010    3.091e+08
Name: United States, Length: 51, dtype: float64

In [67]:
populations = populations[["Australia", "United Kingdom", "China"]]

In [68]:
populations.head()

Country Name,Australia,United Kingdom,China
1960,1.028e+07,5.237e+07,6.671e+08
1961,1.048e+07,5.281e+07,6.603e+08
1962,1.074e+07,5.329e+07,6.658e+08
1963,1.095e+07,5.362e+07,6.823e+08
1964,1.117e+07,5.399e+07,6.984e+08


In [69]:
# Move data n-spaces down
n = 3
populations.shift(n).head()

Country Name,Australia,United Kingdom,China
1960,NaN,NaN,NaN
1961,NaN,NaN,NaN
1962,NaN,NaN,NaN
1963,1.028e+07,5.237e+07,6.671e+08
1964,1.048e+07,5.281e+07,6.603e+08


Note that if you are using a TimeSeriesIndex (coming up), you can specify a period to say something like "move this three days forward".

In [70]:
# Percentage increase from previous row
populations.pct_change()

Country Name,Australia,United Kingdom,China
1960,NaN,NaN,NaN
1961,0.020,0.008,-0.010
...,...,...,...
2009,0.021,0.007,0.005
2010,0.017,0.007,0.005


In [71]:
# Absolute difference to previous row
populations.diff()

Country Name,Australia,United Kingdom,China
1960,NaN,NaN,NaN
1961,206523.0,434000.0,-6.740e+06
...,...,...,...
2009,453200.0,408049.0,6.725e+06
2010,377100.0,417191.0,6.920e+06


### Exercises:

1. Which country had the highest percentage change over 10 years? When did this happen?
2. Which country had the highest absolute growth in one year?
3. Which country had the most stable growth rate during this period? Use standard deviation as a measure of stability (unless you have another metric).

In [35]:
# See solutions/country_changes.py